In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

### **Voting Data**

#### Reading and Initial Filtering/Cleaning of the Data

In [ ]:
# Reads the data
votacao_secao_2022 = pd.read_csv(
    "./data/votacao_secao_2022_PR.csv", sep=";", encoding="latin-1"
)

In [ ]:
# Filters only the desired data

## DS_CARGO must be DEPUTADO ESTADUAL
votacao_secao_2022 = votacao_secao_2022[
    votacao_secao_2022["DS_CARGO"] == "DEPUTADO ESTADUAL"
]

## Only these columns are of interest
columns = [
    "CD_MUNICIPIO",
    "NM_MUNICIPIO",
    "NR_ZONA",
    "NR_SECAO",
    "NR_VOTAVEL",
    "NM_VOTAVEL",
    "QT_VOTOS",
    "SQ_CANDIDATO",
    "NR_LOCAL_VOTACAO",
    "NM_LOCAL_VOTACAO",
    "DS_LOCAL_VOTACAO_ENDERECO",
]
votacao_secao_2022 = votacao_secao_2022[columns]

In [ ]:
# Checks if there are any missing values
votacao_secao_2022[votacao_secao_2022.isna().any(axis=1)]

In [ ]:
# Checks for missing values based on the database's encoding

## Text values
text_columns = [
    "NM_MUNICIPIO",
    "NM_VOTAVEL",
    "NM_LOCAL_VOTACAO",
    "DS_LOCAL_VOTACAO_ENDERECO",
]
print(
    "Missing Text Values:",
    sum((votacao_secao_2022[text_columns] == "#NULO").any(axis=1))
    + sum((votacao_secao_2022[text_columns] == "#NE").any(axis=1)),
)

## Numeric values
numeric_columns = [
    "CD_MUNICIPIO",
    "NR_ZONA",
    "NR_SECAO",
    "NR_VOTAVEL",
    "QT_VOTOS",
    "SQ_CANDIDATO",
    "NR_LOCAL_VOTACAO",
]
print(
    "Missing Numeric Values:",
    sum((votacao_secao_2022[numeric_columns] < 0).any(axis=1)),
)

In [ ]:
# Showing which columns have missing values
(votacao_secao_2022[numeric_columns] < 0).any(axis=0)

In [ ]:
# Showing entries with missing numeric columns
votacao_secao_2022[(votacao_secao_2022[numeric_columns] < 0).any(axis=1)]

In [ ]:
# Filters out missing values
votacao_secao_2022 = votacao_secao_2022[
    (votacao_secao_2022[numeric_columns] >= 0).all(axis=1)
]

In [ ]:
# Database characteristics
votacao_secao_2022.shape

In [ ]:
# Saving the processed data
votacao_secao_2022.to_csv(
    "./processed_data/resultados_secao.csv", index=False, header=False, sep=";"
)

#### Exploratory Analysis

In [ ]:
# Prints the unique values of each column
for column in votacao_secao_2022.columns:
    print(f"### {column} ###")
    print()

    values, counts = np.unique(votacao_secao_2022[column], return_counts=True)
    unique = dict(zip(values, counts))
    for value, count in unique.items():
        print(str(value) + ": " + str(count))

    print()
    print()

In [ ]:
# Plots the top values of some columns
columns = [
    "NM_MUNICIPIO",
    "NM_VOTAVEL",
    "NM_LOCAL_VOTACAO",
    "DS_LOCAL_VOTACAO_ENDERECO",
]
for column in columns:
    values, counts = np.unique(votacao_secao_2022[column], return_counts=True)
    unique = dict(zip(values, counts))
    unique = dict(sorted(unique.items(), key=lambda item: item[1]))

    # Plotting the bar graph
    fig, ax = plt.subplots()
    fig.set_size_inches(16, 10)

    ax.barh(list(unique.keys())[:10], list(unique.values())[:10])

    ax.grid(alpha=0.25)
    ax.set_title(column)
    ax.set_xlabel("N° of Occurences")
    ax.set_ylabel("Value")

    plt.show()
    plt.close()

In [ ]:
# Checks the distribution of the numeric columns
votacao_secao_2022.describe()

In [ ]:
# Shows most voted candidates
most_voted = (
    votacao_secao_2022[["SQ_CANDIDATO", "NM_VOTAVEL", "QT_VOTOS"]]
    .groupby(["SQ_CANDIDATO", "NM_VOTAVEL"], as_index=False)
    .sum()
    .sort_values("QT_VOTOS", ascending=True)
)

# Plotting the bar graph
fig, ax = plt.subplots()
fig.set_size_inches(16, 10)

ax.barh(most_voted["NM_VOTAVEL"][-10:], most_voted["QT_VOTOS"][-10:])

ax.grid(alpha=0.25)
ax.set_title("Most Voted Candidates")
ax.set_xlabel("Votes")
ax.set_ylabel("Candidate")

plt.show()
plt.close()

In [ ]:
# Plotting the bar graph
fig, ax = plt.subplots()
fig.set_size_inches(16, 10)

ax.hist(most_voted["QT_VOTOS"])

ax.grid(alpha=0.25)
ax.set_title("Distribution of Votes per Candidate")
ax.set_xlabel("Votes")
ax.set_ylabel("Quantity")

plt.show()
plt.close()

In [ ]:
# Shows the distribution of votes per city/zone/section
votes_per_city = (
    votacao_secao_2022[["CD_MUNICIPIO", "QT_VOTOS"]]
    .groupby("CD_MUNICIPIO", as_index=False)
    .sum()
    .sort_values("QT_VOTOS", ascending=True)
)
votes_per_zone = (
    votacao_secao_2022[["NR_ZONA", "QT_VOTOS"]]
    .groupby("NR_ZONA", as_index=False)
    .sum()
    .sort_values("QT_VOTOS", ascending=True)
)
votes_per_section = (
    votacao_secao_2022[["NR_SECAO", "QT_VOTOS"]]
    .groupby("NR_SECAO", as_index=False)
    .sum()
    .sort_values("QT_VOTOS", ascending=True)
)

# Plotting the bar graph
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(24, 6)

ax[0].hist(votes_per_city["QT_VOTOS"])
ax[0].grid(alpha=0.25)
ax[0].set_title("Distribution of Votes per City")
ax[0].set_xlabel("Votes")
ax[0].set_ylabel("Quantity")

ax[1].hist(votes_per_zone["QT_VOTOS"])
ax[1].grid(alpha=0.25)
ax[1].set_title("Distribution of Votes per Zone")
ax[1].set_xlabel("Votes")
ax[1].set_ylabel("Quantity")

ax[2].hist(votes_per_section["QT_VOTOS"])
ax[2].grid(alpha=0.25)
ax[2].set_title("Distribution of Votes per Section")
ax[2].set_xlabel("Votes")
ax[2].set_ylabel("Quantity")

plt.show()
plt.close()

### **Census Data**

#### Reading and Initial Filtering/Cleaning of the Data

In [ ]:
# Reads the data
censo_2022 = pd.read_csv(
    "./data/br_ibge_censo_2022_municipio.csv", sep=",", encoding="utf-8"
)
censo_2022.shape

In [ ]:
censo_2022.columns

In [ ]:
# Filters only the desired data

## sigla_uf must be PR
censo_2022 = censo_2022[censo_2022["sigla_uf"] == "PR"]

## Only these columns are of interest
columns = [
    "id_municipio",
    "domicilios",
    "populacao",
    "area",
    "taxa_alfabetizacao",
    "idade_mediana",
    "razao_sexo",
    "indice_envelhecimento",
]
censo_2022 = censo_2022[columns]

censo_2022.shape

In [ ]:
# Checks if there are any missing values
censo_2022[censo_2022.isna().any(axis=1)]

In [ ]:
# Saving the processed data
censo_2022.to_csv("./processed_data/censo_mun.csv", index=False, header=False, sep=";")

In [ ]:
censo_2022.dtypes

#### Exploratory Analysis

In [ ]:
# Aggregation metrics for numerical values
num_columns = [
    "domicilios",
    "populacao",
    "area",
    "taxa_alfabetizacao",
    "idade_mediana",
    "razao_sexo",
    "indice_envelhecimento",
]
censo_2022[num_columns].describe()

### **Census Sector Data**

In [ ]:
# Reads the data
censo_sector_2022 = pd.read_csv(
    "./data/br_ibge_censo_2022_setor.csv", sep=",", encoding="utf-8"
)
censo_sector_2022.shape

In [ ]:
censo_sector_2022.columns

In [ ]:
# Filters only the desired data

## Only these columns are of interest
columns = [
    "id_municipio",
    "id_setor_censitario",
    "pessoas",
    "domicilios",
    "media_moradores_domicilios",
    "area",
    "geometria",
]
censo_sector_2022 = censo_sector_2022[columns]

censo_sector_2022.shape

In [ ]:
# Checks if there are any missing values
censo_sector_2022[censo_sector_2022.isna().any(axis=1)]

In [ ]:
# Saving the processed data
censo_sector_2022.to_csv(
    "./processed_data/censo_sec.csv", index=False, header=False, sep=";"
)

In [ ]:
censo_sector_2022.dtypes

### **Voting Places Data**

In [ ]:
# Reads the data
voting_places = pd.read_csv(
    "./data/geometrias_votacao_2022.csv", sep=",", encoding="utf-8"
)

In [ ]:
# Filters only the desired data

## Only these columns are of interest
columns = ["id_municipio", "zona", "secao", "melhor_urbano"]
voting_places = voting_places[columns]

In [ ]:
# Checks if there are any missing values
voting_places[voting_places.isna().any(axis=1)]

In [ ]:
# Drops missing values
voting_places = voting_places.dropna()

In [ ]:
voting_places.shape

In [ ]:
votacao_secao_2022[["NR_ZONA", "NR_SECAO"]].drop_duplicates()

In [ ]:
# Saving the processed data
voting_places.to_csv(
    "./processed_data/mun_zone_section_location.csv", index=False, header=False, sep=";"
)

### **RAIS**

In [ ]:
# Reads the data
rais = pd.read_csv("./data/rais.csv", sep=",", encoding="utf-8")

In [ ]:
# Checks if there are any missing values
rais[rais.isna().any(axis=1)]

In [ ]:
# Saving the processed data
rais.to_csv("./processed_data/rais.csv", index=False, header=False, sep=";")

In [ ]:
rais.dtypes

### **Extra**

In [ ]:
# Reads the data
extra = pd.read_csv("./data/conectividade.csv", sep=",", encoding="utf-8")

In [ ]:
# Checks if there are any missing values
extra[extra.isna().any(axis=1)]

In [ ]:
# Saving the processed data
extra.to_csv("./processed_data/extra.csv", index=False, header=False, sep=";")

In [ ]:
extra.dtypes

### **Building the Database**

#### Opening Connection

In [ ]:
# Psycopg2
import psycopg2

conn = psycopg2.connect(
    f"dbname={db} user={user} password={password} host={host} port={port}"
)
cur = conn.cursor()

In [ ]:
# SQL Alchemy
from sqlalchemy import create_engine

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

#### resultados_secao

In [ ]:
# Executes changes
cur.execute(
    """
    CREATE TABLE resultados_secao (
        cd_municipio int,
        nm_municipio varchar,
        nr_zona int,
        nr_secao int,
        nr_votavel int,
        nm_votavel varchar,
        qt_votos int,
        sq_candidato bigint,
        nr_local_votacao int,
        nm_local_votacao varchar,
        ds_local_votacao_endereco varchar
    );
    """
)

# Applies changes
conn.commit()

In [ ]:
# Executes changes
f = open("./processed_data/resultados_secao.csv", "r")
cur.copy_from(f, table="resultados_secao", sep=";")
f.close()

# Applies changes
conn.commit()

#### censo_mun

In [ ]:
# Executes changes
cur.execute(
    """
    CREATE TABLE censo_mun (
        id_municipio int,
        domicilios int,
        populacao int,
        area int,
        taxa_alfabetizacao float,
        idade_mediana int,
        razao_sexo float,
        indice_envelhecimento float
    );
    """
)

# Applies changes
conn.commit()

In [ ]:
# Executes changes
f = open("./processed_data/censo_mun.csv", "r")
cur.copy_from(f, table="censo_mun", sep=";")
f.close()

# Applies changes
conn.commit()

#### censo_sec

In [ ]:
# Executes changes
cur.execute(
    """
    CREATE TABLE censo_sec (
        id_municipio int,
        id_setor_censitario bigint,
        pessoas int,
        domicilios int,
        media_moradores_domicilios float,
        area float,
        geom geography
    );
    """
)

# Applies changes
conn.commit()

In [ ]:
# Executes changes
f = open("./processed_data/censo_sec.csv", "r")
cur.copy_from(f, table="censo_sec", sep=";")
f.close()

# Applies changes
conn.commit()

#### mun_zona_sec_local

In [ ]:
# Executes changes
cur.execute(
    """
    CREATE TABLE mun_zona_sec_local (
        id_municipio int,
        zona int,
        secao int,
        geom geography
    );
    """
)

# Applies changes
conn.commit()

In [ ]:
# Executes changes
f = open("./processed_data/mun_zone_section_location.csv", "r")
cur.copy_from(f, table="mun_zona_sec_local", sep=";")
f.close()

# Applies changes
conn.commit()

#### rais

In [ ]:
# Executes changes
cur.execute(
    """
    CREATE TABLE rais (
        ano int,
        sigla_uf varchar,
        id_municipio int,
        tipo_vinculo int,
        vinculo_ativo_3112 int,
        tipo_admissao int,
        tempo_emprego float,
        quantidade_horas_contratadas int,
        valor_remuneracao_media_sm float,
        valor_remuneracao_dezembro_sm float,
        cbo_2002 int,
        cnae_2 int,
        cnae_2_subclasse int,
        idade int,
        grau_instrucao_apos_2005 int,
        nacionalidade int,
        sexo int,
        raca_cor int,
        indicador_portador_deficiencia int,
        tipo_deficiencia int
    );
    """
)

# Applies changes
conn.commit()

In [ ]:
# Executes changes
f = open("./processed_data/rais.csv", "r")
cur.copy_from(f, table="rais", sep=";")
f.close()

# Applies changes
conn.commit()

#### extra

In [ ]:
# Executes changes
cur.execute(
    """
    CREATE TABLE extra (
        ano int,
        sigla_uf varchar,
        id_municipio int,
        ibc float,
        cobertura_pop_4g5g float,
        fibra int,
        densidade_smp float,
        hhi_smp int,
        densidade_scm float,
        hhi_scm int,
        adensamento_estacoes float
    );
    """
)

# Applies changes
conn.commit()

In [ ]:
# Executes changes
f = open("./processed_data/extra.csv", "r")
cur.copy_from(f, table="extra", sep=";")
f.close()

# Applies changes
conn.commit()

#### geometries

In [ ]:
gdf = gpd.read_file("./data/PR_Municipios_2024.shp")
gdf.to_postgis("geo_mun", engine, if_exists="replace", index=False)

gdf = gpd.read_file("./data/PR_RG_Imediatas_2024.shp")
gdf.to_postgis("geo_reg_ime", engine, if_exists="replace", index=False)

gdf = gpd.read_file("./data/PR_RG_Intermediarias_2024.shp")
gdf.to_postgis("geo_reg_int", engine, if_exists="replace", index=False)

#### Closing Connection

In [ ]:
# Finilizes the connection
cur.close()
conn.close()
engine.dispose()